In [ ]:
%matplotlib inline
import pandas as pd
import operator
import psycopg2
import pylab
import numpy as np
import datetime
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.patches as mpatches
from scipy.stats.stats import pearsonr
import matplotlib.lines as mlines
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from statsmodels.tsa import stattools
import statsmodels.api as sm
import scipy
import random
import seaborn as sns
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
import re
from collections import OrderedDict
import statsmodels.api as sm
from scipy import stats
import statsmodels
from statsmodels.graphics.api import qqplot

mpl.rcdefaults()
pd.options.display.mpl_style = 'default'

In [ ]:
# Read database parameters from default_profile
dbitems = {}
with open('default_profile') as f:
    for line in f.readlines():
        item = line.split(" ")[1].split("=")
        dbitems[item[0]] = item[1].strip()
        
# Connect to database with psycopg2
try:
    conn = psycopg2.connect("dbname='%s' user='%s' host='%s' password='%s'"%(dbitems['PGDATABASE'],dbitems['PGUSER'],dbitems['PGHOST'],dbitems['PGPASSWORD']))
except:
    print "Unable to connect to the database"
    
# Connect to database with sqalchemy
conn_sqlalch = create_engine('postgresql+psycopg2://%s:%s@%s/%s'%(dbitems['PGUSER'],dbitems['PGPASSWORD'],dbitems['PGHOST'],dbitems['PGDATABASE']))

# Function Bank

In [ ]:
def get_sec(t):
    #simply get number of seconds
    return t.seconds

In [ ]:
def switch_to_M(unit):
    #change old coding of medics to uniform
    if len(unit) == 3 and unit[0] in ['A','R']:
        unit = 'M' + unit[1:]
    return unit

# Code

In [ ]:
inc_df = pd.read_sql_query("SELECT i_eventnumber, iti_typeid, iti_typetext FROM clean_csvs.dbo_rfirehouseincident", conn)
cad_df = pd.read_sql_query("SELECT i_eventnumber, iiu_tdispatch, iiu_tclear,iiu_tarrive, pun_unitid FROM clean_csvs.dbo_rfirehouseapparatus", conn)

In [ ]:
cad_df = cad_df.dropna()

In [ ]:
cad_df['unit_type'] = cad_df['pun_unitid'].apply(lambda x: x[0])
cad_df['len_unit_type'] = cad_df['pun_unitid'].apply(lambda x: len(x))

In [ ]:
medic_df = cad_df[cad_df['unit_type'].isin(['M','A','R'])]
medic_df = medic_df[medic_df['len_unit_type'] == 3]
medic_df = medic_df.drop('unit_type', 1)
medic_df = medic_df.drop('len_unit_type', 1)

In [ ]:
#get only date for medic dataframe
medic_df['pure_date'] = medic_df['iiu_tdispatch'].apply(lambda x: datetime.datetime(x.year, x.month, x.day))

# Time Series of Medic Demand by Hour

In [ ]:
#get column for pure date only with hour
medic_df['pure_date_with_hour'] = medic_df['iiu_tdispatch'].apply(lambda x: datetime.datetime(x.year, x.month, x.day, x.hour))

In [ ]:
medic_df = medic_df.sort('iiu_tdispatch')

In [ ]:
#groupby pure date with hour
gb_time = medic_df.groupby(['pure_date_with_hour'])

In [ ]:
#make dataframe of times and demand during that time
times = []
demand = []
day_gap = datetime.timedelta(1)
hour_gap = datetime.timedelta(0,0,0,0,0,1)
for name,group in gb_time:
    temp_time = name
    while len(times) > 0 and temp_time != times[-1] + hour_gap:
        times.append(times[-1] + hour_gap)
        demand.append(0)
    times.append(name)
    demand.append(len(group))

In [ ]:
#create the time series dataframe
ts_df = pd.DataFrame()
ts_df['Time'] = times
ts_df['Demand'] = demand
ts_df.index = ts_df['Time']
ts_df = ts_df.drop('Time',1)
index = 0
key_time = datetime.datetime(2007,11,30)
for item in ts_df.index:
    if item >= key_time:
        break
    index += 1
ts_df = ts_df[index:]

In [ ]:
ts_df_train = ts_df[:-1]
ts_df_test = ts_df[-1:]

In [ ]:
#plot acf and pacf of time series
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(ts_df.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(ts_df.values.squeeze(), lags=40, ax=ax2)

In [ ]:
#try an ARMA 1,1
arma_order = (1,1)
arma_mod = sm.tsa.ARMA(ts_df_train, (1,1)).fit()

In [ ]:
#print parameters of ARMA fit
print arma_mod.params

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
ax = arma_mod.resid.plot(ax=ax);


In [ ]:
#get residuals
resid = arma_mod.resid

In [ ]:
plt.hist(resid, bins = 100)

In [ ]:
#plot residuals with acf and pacf
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(resid.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(resid, lags=40, ax=ax2)

In [ ]:
#make plot of medical transport units dispatched
ts_df_less = ts_df_train[:-1]

mpl.rcdefaults()

ax = ts_df.plot(figsize=(12,8))

#fc_vals = arma_mod.predict(start = str(datetime.datetime(2016,6,17,12)), end = str(datetime.datetime(2016,6,17,14)))

rng = pd.date_range('11/30/2007', periods=len(ts_df_less), freq='H')
ts = pd.Series(arma_mod.fittedvalues.tolist()[1:], index=rng)

int_predict_demand = arma_mod.fittedvalues
for i,pred in enumerate(arma_mod.fittedvalues):
    int_predict_demand[i] = round(pred)

#ax = arma_mod.fittedvalues.plot(ax=ax, style='r', label='ARMA(1,1) Fit');

ax = ts.plot(ax=ax, style='r', label='Fit')

#ax = fc_vals.plot(ax=ax, style = 'g', label = 'Forecast', linewidth = 2)

ax.legend(loc=1)

ax.set_xlim(datetime.datetime(2016,6,11), datetime.datetime(2016,6,18))
ax.set_xticks(pd.date_range('6/11/2016', periods=7, freq='D'))

ax.set_ylabel('Ambulances Dispatched', fontsize = 18)
ax.set_xlabel('Time', fontsize = 18)

ax.set_ylim(0,25)

plt.tight_layout()

plt.savefig("ARMA_1_1_Hourly.pdf")

# Time Series of Medic Demand by Day

In [ ]:
#group now by just pure date
gb_time = medic_df.groupby(['pure_date'])

In [ ]:
#create same time series as bevore but now by day
times = []
demand = []
day_gap = datetime.timedelta(1)
for name,group in gb_time:
    temp_time = name
    while len(times) > 0 and temp_time != times[-1] + day_gap:
        times.append(times[-1] + day_gap)
        demand.append(0)
    times.append(name)
    demand.append(len(group))

In [ ]:
#rerun the same time series dataframe creation
ts_df = pd.DataFrame()
ts_df['Time'] = times
ts_df['Demand'] = demand
ts_df.index = ts_df['Time']
ts_df = ts_df.drop('Time',1)
index = 0
key_time = datetime.datetime(2007,11,30)
for item in ts_df.index:
    if item >= key_time:
        break
    index += 1
ts_df = ts_df[index:-1]

In [ ]:
#take first difference of time series to ensure stationarity
ts_df_fd = ts_df
start = ts_df['Demand'][0]
ts_df_fd['FD_Demand'] = ts_df['Demand'] - ts_df['Demand'].shift()
ts_df_fd = ts_df_fd.drop('Demand',1)
ts_df_fd = ts_df_fd[1:]

In [ ]:
ts_df_fd

In [ ]:
#plot first difference acf and pacf
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(ts_df_fd.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(ts_df_fd.values.squeeze(), lags=40, ax=ax2)

In [ ]:
#generate ARMA model for day only
arma_order = (1,1)
arma_mod = sm.tsa.ARMA(ts_df_fd, (arma_order[0], arma_order[1])).fit()

In [ ]:
print arma_mod.params

In [ ]:
#plot residuals
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
ax = arma_mod.resid.plot(ax=ax);


In [ ]:
resid = arma_mod.resid

In [ ]:
plt.hist(resid, bins = 100)

In [ ]:
#plot residual acf and pacf
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(resid.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(resid, lags=40, ax=ax2)

In [ ]:
#predict demand as integers only
int_predict_demand = arma_mod.fittedvalues
for i,pred in enumerate(arma_mod.fittedvalues):
    int_predict_demand[i] = round(pred)

In [ ]:
#get residuals once we convert to integer predictions
actual_dem = [start]
pred_dem = [start]
for item in ts_df_fd['FD_Demand']:
    actual_dem.append(actual_dem[-1] + item)
for item in arma_mod.fittedvalues:
    pred_dem.append(pred_dem[-1] + item)
rng = pd.date_range('12/01/2007', periods=len(ts_df_fd)+1, freq='D')
ts_actual = pd.Series(actual_dem, index=rng)
ts_pred = pd.Series(pred_dem, index=rng)

In [ ]:
#plot graph of medic demand change
ax = ts_df_fd.plot(figsize=(12,8), label = 'FD of Medic Demand')

fc_vals = arma_mod.predict(start = str(datetime.date(2016,6,15)), end = str(datetime.date(2016,6,17)))

rng = pd.date_range('12/01/2007', periods=len(ts_df_fd), freq='D')
ts = pd.Series(arma_mod.fittedvalues, index=rng)

int_predict_demand = arma_mod.fittedvalues
for i,pred in enumerate(arma_mod.fittedvalues):
    int_predict_demand[i] = round(pred)

ax = ts.plot(ax=ax, style='r', label='Fit')

#ax = fc_vals.plot(ax=ax, style = 'g', label = 'Forecast')

ax.legend();

ax.set_xlabel('Time', fontsize = 18)
ax.set_ylabel('Change in Ambulances Dispatched', fontsize = 18)

ax.set_xlim(datetime.datetime(2016,1,1), datetime.datetime(2016,7,1))

plt.savefig('FD_Demand_daily.pdf')

# Time Series of Medic Demand by Area

In [ ]:
#for deepdive track two ambulances

id_35 = pd.read_sql_query("select i_eventnumber from clean_csvs.dbo_iincident where ST_Contains('0106000020E6100000010000000103000000010000002E020000C0B5CB6C9B2355C0D34B916C6193434068B82C069C2355C0A8E010F5569343405B96B1399C2355C07DC8FB1853934340A6D426989C2355C063692E9C4C9343407A17F86D9D2355C00FC3597A3D9343407E0165BE9D2355C0A9992DF1379343408499546BA92355C05CC21D1038934340A29B40EBB12355C0ECD23FF0389343404B80AB22BE2355C003F58D373A934340143E6F25C12355C0FF8EED823A9343406D20D9F7C32355C09A2C02D03A93434060857A4AC62355C09AE1120D3B9343401A44DE32C92355C0F39C60593B93434017023AA8CC2355C0952635B53B9343404AD59D90CF2355C0E9AB7F013C9343405791FE20D22355C01004114A3C9343403F6E44B1D42355C0A5C9C98B3C93434013F79E57D72355C01D6490D33C9343408E9815F5D92355C05B3FD0143D934340588A3AA4DC2355C08BD9475C3D934340D1CE4757E02355C079F82DC03D9343409C8010D1EB2355C0A7D433EA3E934340901FC5E9F12355C02CEF525D3F934340D77CBF35F52355C00C07BD8429934340BDA29E9AF92355C0B8919DD1119343407C9D512BFC2355C00638F722049343408899B39CFD2355C0FFB0E1ECFB924340F94F84B00B2455C0D579A1B7AF924340A906FCD7092455C03CDA4D79A19243401AB00478062455C0FE57AF8F989243405089EDC1032455C0BF5789B8959243404442F955F72355C0B8EEF99E95924340919B368EF52355C01F4D582B95924340CA3CD657F32355C03AE3D5199492434017958D3EF12355C0B3569394929243400C1477B1EF2355C07F98C9EA909243400A697D26E92355C0846D55C5889243405B3D4529E72355C022C012BBF6914340256505EBE62355C0DAA54C11F4914340BE3F5673E22355C0D9179FEFD8914340C58010E1DE2355C016FAFE90AF914340B9D327BFDB2355C06D4702319D914340E7DBF1DBDA2355C0E6FBF82398914340A4ABAA70D92355C088F70A2C94914340B0D3A4E4D22355C0041646418291434037183831D92355C04C1165947C914340D1907C28DE2355C01B01F80378914340B0EF52A6DF2355C097BDCCA676914340CC47580EE52355C0E7159FAB7191434080944D1BE72355C05F0C3AC76F914340EF796BBFE82355C0E615DB4F6E9143409537DCF4E42355C06B31C6FD6B914340650BCD53E42355C04F78E1E2429143401BA7F93EE22355C003F8A75CB490434008A937B3E12355C04708921A8F90434098D00EC84E2455C033D016C18B90434028CD6F93512455C0B3A526118E90434012E40459532455C07085FDD090904340670E4909552455C02649BD6492904340C9856F69572455C0AC5F158494904340EC7DDE985A2455C0F235A84F979043401D3788D65C2455C03CDD7E1C99904340081783BF5E2455C02F7229879A904340ABD6E28E602455C00485A2DB9B904340928D4B39622455C02E4844F99C904340E0FDCD60642455C06B0E11639E904340F2D304F2652455C07C261B279F9043403403AF8E672455C09E50EEF89F904340092BA496692455C0163338DBA0904340C1B2AC586B2455C086441190A1904340127B8A3C6D2455C084914934A2904340A2959E1C6F2455C0F149C66CA2904340945FF923712455C0C3EC596BA29043409E856705742455C089D5793CA29043401088E131772455C000FB36FDA19043409C47572F792455C0EEFD79D6A190434066D23F547B2455C07B0579A7A1904340CDC762017E2455C042FF9B73A1904340E46C6C5F802455C0DBE18F42A19043403FAB8822832455C0DEF6E80DA1904340F58F699C982455C0407279379F90434039D1C5EC982455C06057328B989043406E77D944992455C0321DA8C0929043405C701FE4992455C02ADB7C8988904340D7DB63719A2455C0C20E72D882904340AE126C549E2455C0F1BCC6B57F904340B8948148A42455C0ABF0C1A07A904340941ABE44A72455C0D4792B1678904340EF386679AA2455C043529C5975904340085BD527AD2455C0598CB47E7590434020225A55B02455C0EE924E6975904340B40DB992B62455C0FDE77C7D7B9043404F77E81AB92455C008DA5A617E904340317CE712BC2455C0DE24EEC281904340E8FDEA63BF2455C08CB8260F87904340867C45F6C12455C08A4C84358B904340C679E78BC62455C0DB007FAE929043407D64B31AC92455C052DAB6F79290434065C1252CD12455C0FE17FD2E969043400B6BAA31D92455C06A080A809990434080BAB6F5DE2455C0CCD2277BA090434067D6FFE1E22455C01467629CA59043404DBB8C70E52455C048ECBBF1A890434088ABC814EF2455C095D6DBB1B5904340BA604BD4EE2455C0F9122743BC90434050FD4572EE2455C036D1BB0FC49043406CDC82B1ED2455C0D73F715DD3904340E920548DED2455C09E3F5F01D6904340330B9F39E22455C0262006DCD49043400F911D54DD2455C0C43C6454D49043405DBCD758DC2455C0E53EC708E0904340C715C8CFDB2455C01AE4FDA7EB904340264386D7DA2455C0850A7BE6FE90434093037B9BD92455C0E8A15B7A17914340F399E500D72455C0F622D9991C91434001D13988CF2455C07361866A29914340FED9C32CCB2455C0C1203FE030914340A7D6B057CB2455C0FDCA229E3A9143406F2D50D63E2555C0A24AC18036914340F483CA213C2555C0E9624C9982904340CEAE52C75B2555C0714F049681904340C6EBC3A5AF2555C0003947717E904340B2A4329CAE2555C0449F2C111C904340654A7429C02555C0EA6F4A831E904340D5129002C22555C0924D155E06904340E5EEE6C9CA2555C02447AAB507904340173A4346DE2555C05037B1B90A904340CDC1BFDFEB2555C01A6212E20C9043409443CA39F22555C06CC78FDB0D904340DEB18076FF2555C075EFF8E40F904340C57DC839072655C0A5EB030F11904340BE2E7F2D0D2655C0CBE5015416904340786303B4192655C06C112E7F20904340DD2051021C2655C0505B676021904340A27558CB212655C03196BEE7229043404167B21E232655C0BB820E467A90434041EE3FD3252655C0D7DE822D2E91434079F06DCC6A2655C0BECF2FCE2B91434081DF5242992655C0499A0CFF299143404B71F0499A2655C0EDB5E83E6D914340CCAF51FF9B2655C05DB053DFDD914340EF11FB740F2755C0116407ADD99143406CCA7A26102755C0893ED64606924340AAD87F34122755C0E2030A948D92434062D909F23E2755C02475A6128C924340430FF7A7852755C09346395E89924340537DCDD2872755C0CEE0903105934340B209707E852755C0D9E11AEB0493434062DAA1EF812755C012D87083049343409F4F4EF77E2755C09A70C0210493434083093EF07B2755C0AA5AACBD03934340013030AC792755C094B2277303934340F14204FF762755C0F2AA9519039343403FF5BA2F742755C0C0E2D9BC029343400892BA6E702755C0E3CF154002934340F2A6ACA66D2755C0CE5FA4E30193434050C290626B2755C08F2AA29801934340FFC49E81682755C0C88577390193434043F30464652755C06DC0E8D000934340F68A2505632755C01F4AA97F00934340E39726BC602755C0941262320093434020BB84335E2755C097AD8ADFFF924340CF9E41D75B2755C0E8736792FF924340BD8B3BF6582755C0C8956034FF92434032CE8647562755C00BC5D3DCFE924340770FAD0F532755C0F450F16CFE92434037A0CCB44F2755C0CB5080FDFD924340111685E94C2755C0DD90B4A0FD9243402BD7BB914A2755C086FAE652FD924340158078FD472755C0A8FB3FFDFC9243402F4A262C442755C03497D0AFFD924340585642C3432755C076ABCE7701934340D07EF081432755C0481ADF52069343405D62EA3C432755C02285B73D0B934340135D4E24432755C09C270FED0E934340A452D94A432755C02EA9D1DC13934340A12B4E11432755C08B74923E189343403C7869A1422755C0ECA15BE6209343406B33F442422755C0BDE9764C249343408BADF301422755C027F3F7B028934340611A68423F2755C009469E9028934340F911AC56392755C0093B5AD0279343409BB13999362755C07FF743AA27934340C2815CA4322755C087B1D26227934340872C42D02F2755C04B7796182793434026BAFAB32C2755C080C7D9C32693434073A6BDA9292755C0DE19C46D269343408E6D73DA262755C0AD62AE23269343404B7B0BD3242755C05945BCEA25934340B585360B222755C0FBC7A49B259343409B0854BE1F2755C0970E31602593434058C673D71D2755C082E53D2825934340BD6D74181B2755C08A9D46E024934340FAE31810192755C046EBEAA724934340BBC47250172755C06BBA037824934340D16A5C2C152755C061F32C42249343403401F305132755C0D11A31FC239343407E3D840E112755C0D20E39CC23934340A888B0EC0D2755C0AA6F107423934340BD7030430B2755C0744E9F2E2393434027C302E5082755C0D40437EE2293434082B82BB2062755C05BF8B7B0229343402095791A042755C0A204FA6B22934340CB6AAA68012755C09BB5F72222934340A57C5A6DFF2655C0240304ED2193434091169563FD2655C0CE1C73B52193434014121400FB2655C0E96DBB6F219343400819ECFCF72655C000CE7C2221934340A3054BC3F42655C09258D5CA209343402D269789F12655C05EFF8E6F20934340F2336A2EEF2655C0F1860DBC209343401617D0D9EE2655C0AE6DBDB623934340B2CD00B9EE2655C0C21B725526934340779DED94EE2655C03846F2352993434049F0FF60EE2655C09D86DA112D934340448E3D33EE2655C0843CD28730934340CC31CEC8ED2655C076F1E74B37934340E2CCDD90ED2655C0030178EE3A934340B144106DED2655C085AC99DD3D93434057D8213AED2655C05D3934EC4193434051570710ED2655C0EC2A7E5B45934340E5F03FE8EC2655C0DBD23CC6489343408F1E20BFEC2655C074E0EF034C9343404A08448CEC2655C03DCA23FD4F9343402A4F6346EC2655C047BB953055934340A60EEC13EC2655C0DAAF955E5893434085E9BEC9EB2655C0760E5DA15C934340A825D28EEB2655C070E2AF2F62934340BE95CE9EEB2655C0C803175E689343408B7CCCA2EB2655C0023C31B26B934340F2C23C81EB2655C0D6189B9070934340B08DFB6AEB2655C07CC618C07593434062E3C152EB2655C02D6F97D17C934340E42927CDEB2655C0B007DDFD8093434029677658EC2655C0CD9C249D83934340E32A1346EC2655C0F6B5619686934340D710FC6CEC2655C01DF784988993434092DCA094EA2655C08E2682418D934340240ECDF3E92655C0BC3D346C91934340D7A6851FE92655C095BCDF0A97934340A102CCB0E82655C0606803FB999343402A70F852E82655C03830E1899C934340024479BAE72655C0B3D7B01DA19343409D59906CE72655C04BF706CFA493434029DE90EAE62655C0BB17EE82A893434005CA0467E52655C0F1F19309B39343407C475DC4E42655C0638F0A8FB69343401EB8D930E42655C06DC17143B9934340C537AB92E22655C0882982E1B893434019F313B4E12655C040A9FA6ABC934340BAF13730E12655C0F5ECC70EC0934340F4012397E02655C023D04678C3934340C2215A4FDC2655C0369C8949C393434099CBE919D32655C0C6D3B26AC0934340A39CD109D12655C023F015C6BF934340A998EB87CD2655C0974D31AEBE9343400F75EAA5CA2655C04A6DC3C7BD9343409658D5E1C92655C0792E2E18C4934340EA3D0A6DC92655C01E28F6E9C7934340E27E7DF2C82655C061DC1062CA93434099074D6FC52655C06B1A3451C993434069F683EAC12655C0F92E5E3FC8934340B383EF99BE2655C0442E693DC79343400732E7E1BA2655C039F8FD1BC69343400F5A226BB82655C0BA5D3D5CC5934340D1AFB8A8B52655C02CD37C85C4934340DD7947B3B32655C02CE62624C49343405AFDCDB9B22655C06B9CA330C79343409652403AB22655C0D375F7DACA93434060013174B12655C0B18EBB94D0934340E6A195F3B02655C0164C0666D4934340C51BDE79B02655C0B4084204D89343402933B1D7AF2655C0165012B2DB934340B8B7C711AE2655C0DDA2257CDB93434045FD1A6FAC2655C0DE83AB20D993434046B869C4A82655C0BC7C613DD1934340CC6305B5A62655C0569609C4CE934340F5C400F8A42655C00EE531D3D1934340C0002409A32655C0035EA3AED49343401342788FA12655C0B9088D4FD693434093378986A02655C0A1C410C6D99343409DFA5DBF9F2655C0AC0D8460DC9343403F5AF26C9D2655C098D3A2D5DC934340626AD4FF9C2655C021B4582BE093434005A73DD49C2655C06FC1B665E393434045D7859E9C2655C0A5D5EB5FE793434054598E7B9C2655C0C6C2E0F6E9934340587AB1459C2655C03838C2F3ED93434085FC65139C2655C01BD36BB0F19343404454B9649D2655C04F8AAA3AF39343405D46834F9D2655C0DF586668F7934340D991582D9D2655C03146B351FA9343408E21C4099D2655C022AEDDE9FC934340298101C09C2655C050D0F23F01944340D2C0D0849C2655C0483079BC07944340F598AD399C2655C0E2971FE90E944340A22A64079C2655C0F8ED07BF129443409CB0A8C29B2655C00D5B6EFD1794434072FF2D7C9B2655C0A1723A601D944340ACB1F34F9B2655C0F98FC5C120944340D2F5F4149B2655C0B0BA534525944340773854DA9A2655C00C6E51C1299443409B600AA59A2655C0ABEBDCD32D9443401F4411659A2655C02AFC48B932944340E459702A9A2655C08EA9463537944340E3359DF5992655C03A6E983F3B9443408E582EB5992655C05FFE412D40944340D67C6673992655C0DF8B5DEB44944340F022A550992655C0696E74D347944340A7F67615992655C0861D339C4C944340A3827ABE982655C0CCABADD0539443408953AB8C982655C02317F3C9579443400CF33A59982655C09A8E35EC5B9443407FBBBD28982655C0C343B0F65F944340B5BF40E8972655C07C87B3BF64944340DBDE26B8972655C051F52D50689443404CB59F89972655C0B1D1D5C26B944340E9ACC854972655C016CC71AE6F9443406433B71A972655C0598091017494434014DF05D8962655C017FC94F078944340438E978E962655C042291E5F7E94434062BC2651962655C09C9F3FE882944340075A5B1E962655C08378CC6286944340F53CC1D6952655C05BDCD08D8B9443403633A583952655C0215E46A491944340BABA1B3A952655C08C563E219794434034EC73F2942655C006A1DA699C9443400487D1B7942655C07D106701A19443409F59626B942655C09E735AF0A5944340F41A042B942655C00AA34C21A9944340EC41C5F1932655C08F72953EAD94434008DB2188932655C0C57FC8B2B494434083632F36932655C0EE778B81BA944340897B1FED922655C0D2A04AB5BF94434076223DA6922655C0986238C6C49443400EE18759922655C06090A91AC8944340005C4533902655C0864068E0C89443406EE3C4168E2655C06F7D1E8DC8944340FD5B18788A2655C005F8EC06C8944340C768A244872655C001627C97C79443403BAB0AC4842655C01AF83641C7944340912A8365802655C08C43659EC69443403EC967F57D2655C0D9680E2BC79443407316E57E7F2655C087C86825CB94434041F966B9802655C0472B7CEBCE9443400D9D063E822655C02B4FED54D3944340A9A7BA34852655C0A5591121DA94434068B7CBF9842655C0A9566829DD94434076A2ECEB842655C07861462FE094434098F12E9F842655C0D7E2E4C0E5944340F9659C74842655C060B06EACE89443402233434A7C2655C095DD9FCAE7944340AB7D8D8E7B2655C092F02867F49443402D4B7E08602655C0BC714421F29443401A28A5F45D2655C04B1D51DCF1944340331E267B5A2655C06F9B3F5DF1944340BC40D81E572655C0699123E2F0944340FEFF8AC2532655C0485BD666F094434087F54066502655C0649E5FEBEF944340655FFB094D2655C0414FBF6FEF9443409A3412BD492655C0FC1D4FEEEE944340BF69986C472655C076AFE7A0EE944340E95F5A4B3D2655C01B1DDC2AED94434056318F60382655C01CE0C324EE94434090D7F9CD372655C02D0C13F0F89443403485A9D5362655C0014706BC0A954340D6941062362655C08766030A139543400EFC0331362655C0D79B5FB21695434088EF3FC1352655C02058D6451F954340E2A32E0F352655C0995763222D95434016A8E207342655C01E9B960C339543400EB88AB62F2655C0A685948432954340747898412C2655C007A5921A32954340E24F6600282655C0681F1F9631954340BDA9AB57252655C039CF3C4731954340A9B9145E202655C045A1C3B0309543405494D8D01A2655C0AF7B09FE2F954340D44032490F2655C04FF1A1A52E954340BEA06CC2FD2555C0D9735B852C954340B83E1FBEF62555C0CFE03DA32B954340D0D15A2CF32555C00F09262F2B954340C6C3210FEC2555C0E9254F532A9543400B2CB074E92555C05F8421ED299543401ADD0B4CEA2555C0B1A1AC1016954340FE5C93DAE92555C03FBAF109119543402DA20CF7DD2555C0951BD49B0F95434018E5F93BD72555C053275FC60E954340F9CD7E65CA2555C0F08E4D360D954340C615D725C32555C0B4E25D560C954340CC0D3402C12555C09B5245140C9543400C36C784992555C0BD401A4F0795434035732BA28E2555C0F5BC7D1D06954340FCB5BC668D2555C0BE8E0B8F1195434048AE2F2E8D2555C0351860AF15954340E2E62AC08C2555C0DB577E9C19954340AD75D048852555C02D1694B018954340E2D24DE27F2555C0F38D8706189543402D183C265B2555C08E4C5368139543404D63501A512555C0A0DFF4F412954340A868787C4E2555C057C2FDF44795434055DDC3D34C2555C0328C27E5E99443401CB2279DE62455C0EE43FA45ED9443408D5F8BDFD82455C04D428E23D19443407236C1BBCF2455C09E323D62BE944340959FC619BB2455C03E4BCEFE939443400DCA2F75BB2455C07AAEDF71909443403CA10991BF2455C0D1C93804919443406D907BC3C52455C0D7CDE2C8919443404F16632ACD2455C093E06FB2929443406647B09FD12455C0972EA93B939443403985EFF1D72455C0373CF1DE93944340A61CF231E52455C00C3B1395959443401A7D54BDE92455C063C65324969443401A29D32EF72455C03FD843BF97944340D39B54B2FA2455C0E82125DB8D944340BB16AE3BFF2455C01874A31781944340906BBA87042555C090A3B1CC759443403DEFFC17092555C02E5A03086C944340BA3D90920D2555C04CBDA75862944340AD2348E2112555C008D82BF65894434054A44D9E1B2555C095753D244F944340BD6284F3212555C07A085BD7489443406E19056B292555C0D6F0616041944340D9840EB52A2555C0B589B7EE3D944340CDB0C00F2E2555C01BAB2CE7349443401A0B4B0F322555C0EC08E6962D94434019CEE464332555C029533BB729944340B745A194372555C0BB1DA4B61D944340ADD6D16B3B2555C0EFA51D531E944340F62E9859452555C08D2B24551F944340E880D5D4492555C034BAE6AE1F9443407591D3EE4F2555C01C25846C20944340B866071E5C2555C0E86C78C021944340508E2AB45C2555C0B15469A218944340A4ADC70B5D2555C0293B2853139443400B359F465D2555C0AAA66BB40F9443407A4E75DB5D2555C0DE2D7018069443400C2255675E2555C010E64AE6FD934340996895E45F2555C008743787E593434090CAC38F5E2555C0DBA106ECDE93434035EB429D5C2555C013C7E340D593434014508B5C592555C024DB8CC8C493434007FCB2CB582555C0DB88D342C293434021C21825582555C07F0733CDBE934340BDFFC2F5522555C08ED5671BCA93434031B53A73512555C05CAFA8B5CC934340760BF32B502555C03128068ECE934340EF2752584E2555C0297D4DCBD0934340EA28E1644C2555C0F02085C3D29343400FF8CA544A2555C0F071036ED49343408F190527482555C0DCA23ACED59343405C6B4EBD452555C0779789E3D6934340C6EF833E432555C0A03247A2D79343403FD5D9003A2555C086D96FAED8934340839CFC1F252555C07F09E934DA934340F0E1ABB11F2555C003BE3210E193434075F9FFFD092555C06FB1C1A8FB934340F2A58976082555C04EACA71CFD9343406066CE80062555C0D94B28C6FE9343407FB4BFE5042555C0841C1CFDFF9343403945681C032555C03B162A2D01944340E649F7DF002555C08F2BE575029443401530F7BCFE2455C074A0597E03944340C517405AFC2455C03644FE65049443400340064AF52455C0B9BE9B2A06944340F3A94C90E82455C04DBC1AAC0994434096A5C4F5D52455C0193EC2360A94434063A09748D32455C0D570214F56934340FF3C38FED22455C01442EBA0429343405B1B273ED12455C034F5EB7CF392434044E22A40D02455C0371F3BB004934340AB4B6899CF2455C097725201119343404F762F59CF2455C0A8F3C71F169343403C0C051ECF2455C0B765A5401A934340A8D664F5CE2455C0536AFC771D9343402302CEA9CE2455C0E654231223934340DC6DBF3FCE2455C07C03CA152B934340F6775BC4CD2455C0A6DE1C5534934340CB44389ACD2455C040AAC32537934340D15F0510CD2455C0043FE84B42934340C3ABDAD7CC2455C03BA576A346934340BB377791CC2455C0A46238744C93434051A6025ACC2455C030D1AFFB509343403FB82EECCB2455C0C795C65E56934340819C5FC65F2455C052C7C28F5A934340DAA663E9612455C0D602A5BAF29343400B1A2CA45C2455C0A3BFD4810494434011E170365A2455C0BF594EA50E9443409AD8CD4D482455C030E31C660F9443404841EE61302455C0A724584010944340B0CC96E2EE2355C0882BED9B129443401745B282F02355C0EAC27C3980944340761D8509D92355C02C5B1EBF9C94434007C8BD45D72355C053A265D99E94434005044E4AD52355C0D6080F11A19443408AC9AF1CD32355C0D88E2D64A394434015A2A2BBD02355C0DDC18AC8A59443408A980FA7CE2355C025BD05C0A794434084193B12CD2355C060256A2DA9944340AE69A01ECB2355C00E2961D8AA9443403BE051F4C82355C0DFD46996AC944340C0087495C62355C0040C0464AE944340C0E9EDA0C32355C0B11A4776B09443404F6AEB32C12355C01F14310AB29443401A8C9259BE2355C0332499BBB3944340FBFD8D0DBC2355C00ECED3FDB4944340E1B7C3BCB92355C0DB798A26B69443404DE2499FB62355C0DAA5BB90B7944340839BB67BB32355C0771F0DD2B89443405A61CA4FB02355C067E722E7B99443406F00C134AD2355C09828ADCBBA9443405A23FEF1AA2355C054EF154CBB944340C6D65CE1A82355C0D5859BADBB9443402A3509CCA62355C08C51BEFABB944340B85AA9B8A42355C0C267B036BC944340A6ED14A5A22355C0FCFAEF64BC9443407B3F29C99F2355C01F27BF85BC944340182ECA709D2355C097A3EB89BC9443404D27FE939A2355C05D76D36EBC9443409CD5F03A982355C00716CB44BC94434005A2352A962355C07289C608BC944340689D754E932355C04E2BE19EBB9443404D1800FB902355C058462630BB94434024DB9DEB8E2355C0826DDDB9BA944340B7492EDE8C2355C05AEF6332BA944340740422B28A2355C03369AF8CB9944340DEDB47758A2355C057D3A265AC9443406E93324D8B2355C057BEC2DE9A944340293FE1638D2355C038C5240B70944340351D97A88D2355C00D27B5E56A944340473EED248E2355C0C0967CCF619443407681C49D8E2355C070280BF7589443409C99142B8F2355C0E42BC9894E944340EBBB09DE8F2355C0D3BF593E41944340BBBE5E5A902355C094F7222838944340CF82B3D2902355C01DAE762D2F944340B6E6224B912355C01D7BA23926944340A871AAC7912355C07CDC1D311D94434061E74609922355C07308D2641894434019CB0236922355C04914F81115944340D58A20BD922355C03AAAFF560B944340B3DBEF43932355C0AAD47C87019443404E85D8CE932355C0BF744BA3F7934340ADBD740D942355C0FEA8FD36F393434087A85B51942355C0A48ACADAED9343402B3CABD3942355C0F3539704E4934340A41A2F52952355C05ED4A057DA934340AF2DF7EA952355C0DB048675CF934340E2F8DDF5962355C0EC330087BA934340A6FC28D1972355C05547B951AA934340FA309EDF972355C0D23A89B2A793434024615620982355C0999D2975A393434004438F85982355C03CD70F409B93434054175065992355C0FCCA407B89934340480692E1992355C0C3D4F1CA7F934340BC1F1F669A2355C0013818F875934340E7E50FCD9A2355C01DF415396E934340C0B5CB6C9B2355C0D34B916C61934340', geom)",conn)
id_19 = pd.read_sql_query("select i_eventnumber from clean_csvs.dbo_iincident where ST_Contains('0106000020E610000001000000010300000001000000B5000000921376A0BF1F55C0BB3B52CB2A90434096F5A147C01F55C0767082F11D9043405E465CF2C01F55C0E781FCA311904340ABB9D13AC21F55C073201517F98F43401023B082C31F55C0236A1A61E08F4340E87018D9C41F55C06ACC810AC88F43403EE06C07C61F55C0DDCDB0FDAE8F43404F7307E2DC1F55C0AE13A7C1B18F43401D7DAF23E51F55C008DDD4B3B28F4340671CACAFED1F55C0CCC7BC89B38F4340BDF09EDFEF1F55C031A9FAFBB38F434035B038E3F11F55C05C79A252B48F43405376C4E7F31F55C08A03FE9EB48F43407A431448F61F55C0E8B9BEEAB48F434028737EABF81F55C0CD88B128B58F4340024EA1B0FA1F55C0937C1051B58F4340AAAF0B6DFC1F55C0AB298A6CB58F4340846CE360FE1F55C03D81AB82B58F4340B23F9C32012055C0D5096D8FB58F43402168905D032055C0E4CEF989B58F4340BC25CB3E052055C02C6B817EB58F43400ED3866A072055C0315EC262B58F4340FAA21E14152055C04D3A96C8B58F4340AF4756C91B2055C0D931D783B68F4340BAADDD28272055C0DC4B610CB88F4340D0911AE9332055C0A11AE0DAB98F4340524AAF93372055C042865A73BA8F4340FEDA021E392055C0356DABF8A18F4340851398293B2055C0C67FF919798F4340ED613EF9402055C073647755148F4340C3B83BBC502055C09FA26DFC128F43401A9EA65B662055C07FE1746E148F4340F24B9E8F772055C054F93C9E168F43408423FBEF852055C00109B212188F4340B6174A95902055C06F86015F1A8F4340D74F6734922055C0736443F01A8F43402EEA42EE952055C02403C91E1C8F4340F6DE616CA82055C0D937FB6B1E8F4340DE28364BAA2055C0BC9F85541E8F4340B12D7C8EB72055C0FCCCECF41F8F4340ED0332A1C02055C0761E48FD218F43402C9F0F61BF2055C0245FF9CC378F434080AA8E1EC02055C0BDC21E91438F4340D485A14CC22055C0806DD211448F43404EB5A546C62055C09A1388424A8F4340918EDD8DCB2055C00D38E22C538F434080046275D12055C040CA05AB5D8F4340A7E05498DD2055C0B49FF0A3718F43400784F141DF2055C06519929C738F4340036B637BE22055C03C88B883818F43403A784508E22055C07BDBF4D2848F4340F471FBA0E02055C0DBC0CBA7878F4340D34F2E42DF2055C08959B77F898F43405F39E691DD2055C0678ECA408B8F4340784FCBB3DB2055C019B8D5188E8F434018D13B8CDA2055C01D4A7891918F43405E4DDED4D82055C0C04AB46F9C8F434081E6F9C1D72055C0B15A6300A38F43406F18CA81D72055C0DB864E5CA78F4340C8FC66B9D82055C0F1A3C1EFAF8F4340A043B9F0D82055C08DB800F0B28F4340AD0C45E3D72055C072C26D8FC58F43405EE04607D82055C0A6CAF804C98F43404F313EE8DF2055C082680740CB8F434055212FE0E62055C0F489632FCC8F43408B293B08EE2055C00F2DAA46CD8F43407ED91EA2EB2055C02E425536DE8F4340B235CF0EEA2055C03A05FB22E88F434006548143F32055C025F650C7EB8F43409B63809FFA2055C070563AD0EE8F434022FA7457FD2055C0775C762CEF8F434002059CC4FF2055C065E67084EF8F4340EF0A0C3C042155C0A1D34990F08F434059C92CC40A2155C08A0AB838F18F4340DC4FA48C282155C04B3E891BF58F4340BCEB6D09352155C0AB5F1277F68F4340EADD21B6442155C0FBD69610F88F434097F25050432155C04A9C21630C90434053286740422155C04317608120904340A3B9B61F422155C08F0C220E24904340393A1CCD412155C021F018CF27904340E8F84D61412155C00EC9F1C52A9043403BF643743E2155C093534E40429043403B4A8DDF3C2155C0B14147F44C90434076E0DB973A2155C09D3BA3CA5C9043407ABFB28C392155C09A71FA9A729043406C1828082C2155C0BDDF28E970904340B4915FCE2C2155C021D7A75CA7904340E145C9222E2155C01F30FA2302914340E5776E86252155C095DE823104914340F23729ED1D2155C06F0810DDF590434018202998072155C0059E725AFA904340DB1C31EAFD2055C01F53CE7010914340AEC1E39CD72055C04318519E0B914340E832DEAABA2055C0BAAD06AB0791434065B7C178B42055C0AE489DD1069143408CDA4155B22055C0A241872128914340130D9FF9B12055C08DF0EB9F2B91434089C539B2B12055C00D0EDB572E914340F40A0D65B12055C09D2B7D4831914340E3B2ED20B12055C099F797FE33914340F35D0ED3B02055C066227F0B379143406A6F2B77B02055C05721D4C53A914340BB8F492DB02055C070F758B63D914340B1966BD5AF2055C09899025641914340E28BA87AAF2055C0CDDD021245914340E476CA10AF2055C070A47A7B49914340EF4637BCAE2055C0B920031B4D91434060146A7BAE2055C02AA7B5D24F9143402455CF3CAE2055C05D98568A5291434016A2FFEDAD2055C09EF474EF559143402935EB23A92055C0A7C3E16D5691434035615A66922055C07038689755914340AFCE92EF9B2055C0C8DECBE6A29143400AB8BEA79C2055C08EE2CD76B4914340653572929C2055C047F05AB1B891434006CC757F9B2055C0C3D245EAD19143404155E4079A2055C0416CA679EF914340CEAB2D68992055C0D6C20384FD914340C78994A6982055C0F069C81C0D9243407E48E9EF972055C081C974E31A9243404F3E6458972055C0C4D43D9027924340B67DA3D4962055C087FC02B230924340F110B2DE962055C0432EA9443392434004201447972055C0AA81F60236924340A729BD5D982055C0D85C7AA6399243402B4678638B2055C06DD3328739924340DE13485F782055C065BA591537924340F25E109F5F2055C06619962734924340F77CF653602055C0D0021625249243405C406F59602055C0B927AB791692434002BAB6345E2055C00E30C14016924340FD90D60B5C2055C00EB8DAFA159243401F5D91DE592055C082FB16B515924340F46EFAB3562055C0AEEF014F15924340EAAEC78B532055C047DB30CA1492434094BE6EBC532055C0A793F19910924340B7B7A601522055C03D704674109243407424FC9B522055C0CFB6AC430592434068AEA275532055C0DFC6DBFEF4914340C18D3958542055C084E3E353E39143405C2E966B472055C065B9B4A6E0914340C09F81BD3B2055C0B384FE79DF914340CF369024252055C077A73C49DD914340CDA34933222055C02F730E65D891434087D13AAD1F2055C04E523A03CF91434028C509B11C2055C0FBB598FEC4914340ABFF96791B2055C00FA4600AC3914340650647A1192055C002E0E0DAC1914340C8E78AC90A2055C00AF5506BBF914340D1F52D0C0B2055C0F4A8D791BB914340BBDB45470E2055C0C57D93E87C914340657DA2880F2055C0611106DE64914340A9656AC8102055C03196249A4D914340C3D81994142055C0520FDE330B9143401E9C0C89152055C08CDD8188F9904340AF4F7EAF112055C0C5829BFBFA904340496641B5FB1F55C0D8C159B902914340637E5760E41F55C0663282C8FF90434091BFF811DA1F55C0CB30525903914340B2232CCDD11F55C017CBD27D0391434045AC608FE21F55C0C785C44EB09043408652A475E71F55C0AC0232369190434066339218E91F55C061C8A1828B904340BA5A88EEEC1F55C028F24A41829043401BFAD685F41F55C058780C2B76904340521AA8A9F51F55C0C042DC33749043407FD87D6CFA1F55C03A6F24036C904340D5B3AED6FF1F55C082A83D1C65904340562A2ECE042055C03BFC7CBA61904340628D47C70B2055C02A914E815A9043400F729D210F2055C0725090BE54904340ED3D47AD152055C0F55D879F47904340ED177BFD1D2055C03F06B666379043402EC5A9D4002055C0D00F845033904340CDFB77B0FA1F55C068DA0F973290434028E6040CF31F55C045A4DD613190434070B85E75E51F55C0BC9BDEE52F904340C615DDCCDE1F55C05C5CAF0E2F90434077BCD52ED41F55C0374ACB932D904340921376A0BF1F55C0BB3B52CB2A904340', geom)",conn)

#below is much of the same time series analysis as above

In [ ]:
df_35 = medic_df[medic_df['i_eventnumber'].isin(id_35['i_eventnumber'])]
df_19 = medic_df[medic_df['i_eventnumber'].isin(id_19['i_eventnumber'])]

In [ ]:
gb_time = df_35.groupby(['pure_date_with_hour'])

In [ ]:
times = []
demand = []
day_gap = datetime.timedelta(1)
hour_gap = datetime.timedelta(0,0,0,0,0,1)
for name,group in gb_time:
    temp_time = name
    while len(times) > 0 and temp_time != times[-1] + hour_gap:
        times.append(times[-1] + hour_gap)
        demand.append(0)
    times.append(name)
    demand.append(len(group))

In [ ]:
ts_df = pd.DataFrame()
ts_df['Time'] = times
ts_df['Demand'] = demand
ts_df.index = ts_df['Time']
ts_df = ts_df.drop('Time',1)
index = 0
key_time = datetime.datetime(2007,11,30)
for item in ts_df.index:
    if item >= key_time:
        break
    index += 1
ts_df = ts_df[index:]

In [ ]:
ts_df_fd = ts_df
ts_df_fd['FD_Demand'] = ts_df['Demand'] - ts_df['Demand'].shift()
ts_df_fd = ts_df_fd.drop('Demand',1)
ts_df_fd = ts_df_fd[1:]

In [ ]:
ts_df_fd_train = ts_df_fd[:-1]
ts_df_fd_test = ts_df_fd[-1:]

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(ts_df_fd.values.squeeze(), lags=40, ax=ax1)


In [ ]:
arma_order = (1,0)
arma_mod = sm.tsa.ARMA(ts_df_fd_train, (arma_order[0], arma_order[1])).fit()

In [ ]:
print arma_mod.params

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
ax = arma_mod.resid.plot(ax=ax);


In [ ]:
resid = arma_mod.resid

In [ ]:
plt.hist(resid, bins = 100)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(resid.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(resid, lags=40, ax=ax2)

In [ ]:
ts_df_fd

In [ ]:
mpl.rcdefaults()

ax = ts_df_fd.plot(figsize=(12,8))

fc_vals = arma_mod.predict(start = str(datetime.datetime(2016,6,17,10)), end = str(datetime.datetime(2016,6,17,12)))

rng = pd.date_range('12/18/2007', periods=len(ts_df_fd), freq='H')
ts = pd.Series(arma_mod.fittedvalues, index=rng)

int_predict_demand = arma_mod.fittedvalues
for i,pred in enumerate(arma_mod.fittedvalues):
    int_predict_demand[i] = round(pred)

#ax = arma_mod.fittedvalues.plot(ax=ax, style='r', label='ARMA(1,1) Fit');

ax = ts.plot(ax=ax, style='r', label='Fit')

ax = fc_vals.plot(ax=ax, style = 'g', label = 'Forecast', linewidth = 2)

ax.legend(loc=1)

ax.set_xlim(datetime.datetime(2016,6,11), datetime.datetime(2016,6,18))
ax.set_xticks(pd.date_range('6/11/2016', periods=7, freq='D'))

ax.set_ylabel('Ambulances Dispatched', fontsize = 18)
ax.set_xlabel('Time', fontsize = 18)

plt.tight_layout()

plt.savefig("ARMA_1_1_Hourly.pdf")

# Annual Medic Units Analysis

In [ ]:
#get dataframe of medic transport unit stats
feature_df = pd.read_sql_query("SELECT i_eventnumber,m_required,trns_to_hosp,m_sent from karen.featured_table_3", conn)
feature_df['m_required'] = feature_df['m_required'].apply(lambda x: int(x))
feature_df['m_sent'] = feature_df['m_sent'].apply(lambda x: int(x))
feature_df['trns_to_hosp'] = feature_df['trns_to_hosp'].apply(lambda x: int(x))
full_df = pd.merge(medic_df, inc_df, how = 'left', on = ['i_eventnumber'])
full_df['year'] = full_df['pure_date'].apply(lambda x:x.year)
full_df = full_df.dropna()

In [ ]:
code_dict = {}
gb_inc = feature_df.groupby('i_eventnumber')
for name,group in gb_inc:
    sent = sum(group['m_sent'])
    trns = sum(group['trns_to_hosp'])
    if sent == 0:
        if trns == 0:
            ret = 0
        else:
            ret = 1
    else:
        if trns == 0:
            ret = 2
        else:
            ret = 3
    code_dict[name] = ret
    
        
    

In [ ]:
feature_df.groupby.sum(['trns_to_hosp', 'm_sent'])

In [ ]:
#track for a few different types of incidents
breath_probs = []
abs_probs = []
pregs_probs = []
for item in full_df['iti_typeid']:
    if item[0:2] == '06' and len(item) == 4:
        breath_probs.append(item)
for item in full_df['iti_typeid']:
    if item[0:2] == '01' and len(item) == 4:
        abs_probs.append(item)
for item in full_df['iti_typeid']:
    if item[0:2] == '24' and len(item) == 4:
        pregs_probs.append(item)
breath_probs = set(breath_probs)
abs_probs = set(abs_probs)
pregs_probs = set(pregs_probs)

In [ ]:
#get df of type in breathing problems
full_df = full_df[full_df['iti_typeid'].isin(breath_probs)]

In [ ]:
#get only dataframe where medic is sent
full_df = full_df[full_df['m_sent'] == 1]

In [ ]:
gb_date = full_df.groupby('pure_date')

In [ ]:
tups = []
for name,group in gb_date:
    #tups.append((name, np.median([i-j for i,j in zip(group['iiu_tarrive'],group['iiu_tdispatch'])])))
    pct = sum(group['trns_to_hosp'])/float(len(group))
    tups.append((name, pct))
    if pct >= 0.5:
        print group
        raise SystemExit

In [ ]:
#get time series of problems 
ts_df = pd.DataFrame()
ts_df['Time'] = [i[0] for i in tups]
ts_df['Units'] = [i[1] for i in tups]
ts_df.index = ts_df['Time']
ts_df = ts_df.drop('Time',1)
index = 0
key_time = datetime.datetime(2008,1,1)
for item in ts_df.index:
    if item >= key_time:
        break
    index += 1
ts_df = ts_df[index:]

In [ ]:
idx = pd.date_range(str(ts_df.index[0]), str(ts_df.index[-1]))
ts_df.index = pd.DatetimeIndex(ts_df.index)
ts_df = ts_df.reindex(idx, fill_value=0)

In [ ]:
rng = pd.date_range(str(ts_df.index[0]), periods=len(ts_df), freq='D')
ts = pd.Series(ts_df['Units'], index=rng)

In [ ]:
yf = 2013
mpl.rcdefaults()
ax = ts.plot(figsize=(12,8), color = 'b')
for item in [2009,2010,2011,2012,2013,2014,2015,2016]:
    ax.axvline(x=datetime.date(item,1,1), color = 'r', linewidth = 2)
#ax.set_xlim(datetime.date(yf,1,1), datetime.date(yf+1,1,1))
#ax.set_ylim(0, 5*np.mean(ts))
pylab.show()

In [ ]:
df_2011 = medic_df[medic_df['pure_date'].apply(lambda x:x.year) == 2008]
gb_date = df_2011.groupby('pure_date')
freq_dict = {}
for name,group in gb_date:
    num = len(set(group['pun_unitid']))
    if num in freq_dict.keys():
        freq_dict[num] += 1
    else:
        freq_dict[num] = 1
    if num > 20:
        print name,set(group['pun_unitid'])